# GooglePlay Banking App 리뷰데이터 수집
- https://play.google.com/store/apps/details?id=viva.republica.toss
- https://play.google.com/store/apps/details?id=com.kakaobank.channel
- https://play.google.com/store/apps/details?id=com.kbankwith.smartbank

------

#### 전체 데이터 몇 만개를 가져오는 것은 의미가 없음
- 모든 리뷰에 날짜가 존재하기 때문에 2000개 정도의 데이터를 최신순으로 가져옴
1. selenium으로 아래의 링크 접속
2. 전체리뷰보기를 찾아 클릭
3. 리뷰창이 뜨면 최신순으로 정렬
4. 마우스 스크롤을 통해 오늘부터 2년전 날짜가 나올때까지 스크롤 진행
5. 리뷰 데이터 수집
    - 사용자 닉네임
    - 별점
    - 리뷰 작성 날짜
    - 리뷰 내용
    - 답변자 닉네임
    - 답변 작성 날짜
    - 답변 내용
6. csv 파일로 저장 -> DB에 저장
7. 수집 코드를 함수화하여 KB, 신한, 하나 리뷰도 저장

In [13]:
driver.quit() 

In [2]:
# 토스 구글플레이스토어 페이지 확인
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start_maximized')
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
url = 'https://play.google.com/store/apps/details?id=viva.republica.toss'
driver.get(url)

In [3]:
# 리뷰 데이터 전체 보기 클릭
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))

WebDriverWait(driver, 10)
path = '#yDmH0d .wkMJlb .qZmL0 c-wiz:nth-child(5) > section > header > div > div:nth-child(2) > button'
driver.find_element(By.CSS_SELECTOR, path).click()

In [6]:
# 리뷰 데이터를 최신순으로 정렬

WebDriverWait(driver, 10)
driver.find_element(By.CSS_SELECTOR, '#sortBy_1 > div.kW9Bj > i').click()

path_new = '#yDmH0d .fysCi.Vk3ZVd .JPdR6b.e5Emjc.ah7Sve.qjTEB > div > div > span:nth-child(2)'
driver.find_element(By.CSS_SELECTOR, path_new).click()

In [54]:
import time

# 리뷰가 담긴 창을 찾아서 JavaScript로 1000px씩 아래로 스크롤
driver.execute_script("document.querySelector('.fysCi.Vk3ZVd').scrollBy(0, 1000)")

In [132]:
# 별점
driver.find_element(By.CSS_SELECTOR,'div.Jx4nYe > div').get_attribute('aria-label').split()[3][0]

'1'

In [141]:
# 리뷰 데이터_사용자 닉네임
driver.find_element(By.CSS_SELECTOR, "#yDmH0d .fysCi.Vk3ZVd .odk6He .RHo1pe .c1bOId .YNR7H .gSGphe .X5PpBb").text

'Young Carlo'

In [119]:
# 리뷰 데이터_사용자 리뷰 등록 날짜
driver.find_element(By.CSS_SELECTOR, "#yDmH0d .fysCi.Vk3ZVd .odk6He .RHo1pe .c1bOId .Jx4nYe").text
# driver.find_element(By.CSS_SELECTOR, "div.Jx4nYe > span.bp9Aid").text

'2025년 4월 13일'

In [104]:
# 리뷰 데이터_사용자 리뷰
driver.find_element(By.CSS_SELECTOR, "#yDmH0d .fysCi.Vk3ZVd .odk6He .RHo1pe .h3YV2d").text

'이제 됐네요'

In [162]:
# 리뷰 데이터_답변자 닉네임
driver.find_element(By.CSS_SELECTOR, ".fysCi.Vk3ZVd .ocpBU .I6j64d").text

'Viva Republica'

In [118]:
# 리뷰 데이터_답변자 답변 작성 날짜
driver.find_element(By.CSS_SELECTOR, "#yDmH0d .fysCi.Vk3ZVd .odk6He .RHo1pe .ocpBU .T6E0ze .I9Jtec").text

'2025년 4월 13일'

In [89]:
# 리뷰 데이터_답변자 답변 내용
driver.find_element(By.CSS_SELECTOR, "#yDmH0d .fysCi.Vk3ZVd .odk6He .RHo1pe .ocpBU .ras4vb").text

'안녕하세요. 임의수님, 토스팀입니다. 불편한 사항이 잘 해소 되셨을까요? 보다 편리한 토스 사용 경험을 하실 수 있도록 노력 하겠습니다. 다른 문의 사항이 있다면 24시간 운영되는 카카오톡(@toss) 또는 고객센터 1599-4905로 문의 부탁드립니다. 감사합니다.'

In [92]:
from datetime import datetime, timedelta

# 리뷰일을 날짜형 데이터로 변경
review_date = driver.find_element(By.CSS_SELECTOR, "#yDmH0d .fysCi.Vk3ZVd .odk6He .RHo1pe .c1bOId .Jx4nYe").text
review_date = review_date.replace('년 ','-').replace('월 ','-').replace('일','')
review_date = datetime.strptime(review_date, '%Y-%m-%d')
review_date

datetime.datetime(2025, 4, 13, 0, 0)

In [71]:
# 오늘부터 한달 전 날짜 확인
today = datetime.today()
twoyrago = today - timedelta(days=30)

In [49]:
review = driver.find_elements(By.CSS_SELECTOR, ".RHo1pe")[-1]
# print(len(review))
end_date = review.find_element(By.CSS_SELECTOR,"div.Jx4nYe > span.bp9Aid").text
end_date = end_date.replace('년 ','-').replace('월 ','-').replace('일','')
end_date = datetime.strptime(end_date, '%Y-%m-%d')
end_date

datetime.datetime(2025, 4, 12, 0, 0)

In [53]:
print(end_date, twoyrago)
end_date < twoyrago

2025-04-12 00:00:00 2024-04-14 10:12:43.788012


False

# 개별 코드 합치기


In [111]:
# 토스 구글플레이스토어 페이지 확인
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start_maximized')
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
url = 'https://play.google.com/store/apps/details?id=viva.republica.toss'
driver.get(url)

# 리뷰 데이터 전체 보기 클릭
WebDriverWait(driver, 10)
path = '#yDmH0d .wkMJlb .qZmL0 c-wiz:nth-child(5) > section > header > div > div:nth-child(2) > button'
driver.find_element(By.CSS_SELECTOR, path).click()

# 리뷰 데이터를 최신순으로 정렬
WebDriverWait(driver, 10)
driver.find_element(By.CSS_SELECTOR, '#sortBy_1 > div.kW9Bj > i').click()
path_new = '#yDmH0d .fysCi.Vk3ZVd .JPdR6b.e5Emjc.ah7Sve.qjTEB > div > div > span:nth-child(2)'
driver.find_element(By.CSS_SELECTOR, path_new).click()

time.sleep(3)

# 오늘부터 한달 전 날짜 확인
today = datetime.today()
end_date = today - timedelta(days=30)
current_date = ""
reviews = ""

while True:
    
    try:
        # 리뷰가 담긴 창을 찾아서 JavaScript로 1000px씩 아래로 스크롤
        driver.execute_script("document.querySelector('.fysCi.Vk3ZVd').scrollBy(0, 1000)")
        time.sleep(1)

        # 리뷰들 중 가장 마지막 데이터 날짜 추출해 날짜 데이터로 반환
        reviews = driver.find_elements(By.CSS_SELECTOR, ".RHo1pe")
        current_date = reviews[-1].find_element(By.CSS_SELECTOR,"div.Jx4nYe > span.bp9Aid").text
        current_date = current_date.replace('년 ','-').replace('월 ','-').replace('일','')
        current_date = datetime.strptime(current_date, '%Y-%m-%d')
        print(f"현재 리뷰 날짜: {current_date}, 리뷰 수: {len(reviews)}", end='\r')
    except:
        reviews = driver.find_elements(By.CSS_SELECTOR, ".RHo1pe")
        print(f"현재 리뷰 날짜: {current_date}, 리뷰 수: {len(reviews)}", end='\r')
    
    if current_date < end_date:
        break
        

In [113]:
print(len(reviews))

440


In [112]:
# !pip install tqdm

In [177]:
from tqdm import tqdm

all_result = []
for review in tqdm(reviews):
    
    # 리뷰일을 날짜형 데이터로 변경
    review_date = review.find_element(By.CSS_SELECTOR, ".c1bOId .Jx4nYe").text
    review_date = review_date.replace('년 ','-').replace('월 ','-').replace('일','')
    review_date = datetime.strptime(review_date, '%Y-%m-%d')

    # 리뷰 데이터_사용자 닉네임
    user_name = review.find_element(By.CSS_SELECTOR, ".YNR7H .X5PpBb").text

    # 별점
    user_score = review.find_element(By.CSS_SELECTOR,'div.Jx4nYe > div').get_attribute('aria-label').split()[3][0]

    # 리뷰 데이터_사용자 리뷰
    user_review = review.find_element(By.CSS_SELECTOR, "div.h3YV2d").text

    # 리뷰 데이터_답변자 답변 작성 날짜
    reply_section = review.find_element(By.CSS_SELECTOR, "div.ocpBU")
    
    try:
        request_date = reply_section.find_element(By.CSS_SELECTOR, "span.I9Jtec").text
        request_date = request_date.replace('년 ','-').replace('월 ','-').replace('일','')
        request_date = datetime.strptime(request_date, '%Y-%m-%d')
    except:
        request_date = None

    # 리뷰 데이터_답변자 닉네임
    request_name = reply_section.find_element(By.CSS_SELECTOR, "span.I6j64d").text
                

    # 리뷰 데이터_답변자 답변 내용
    try:
        reply = reply_section.find_element(By.CSS_SELECTOR, "div.ras4vb").text
    except:
        reply = None
    
    result = (review_date, user_name, user_score, user_review, request_date, request_name, reply)
    all_result.append(result)

result_df = pd.DataFrame(all_result, columns=['리뷰일자', '사용자이름', '별점', '리뷰', '답변일자', '답변자이름', '답변'])
display(result_df)



  0%|                                                                                          | 0/440 [00:00<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.I6j64d"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007F80E3+60707]
	GetHandleVerifier [0x007F8124+60772]
	(No symbol) [0x00620683]
	(No symbol) [0x00668660]
	(No symbol) [0x006689FB]
	(No symbol) [0x0065DD81]
	(No symbol) [0x0068D094]
	(No symbol) [0x0065DCA4]
	(No symbol) [0x0068D2C4]
	(No symbol) [0x006AE824]
	(No symbol) [0x0068CE46]
	(No symbol) [0x0065C5D3]
	(No symbol) [0x0065D424]
	GetHandleVerifier [0x00A3BBC3+2435075]
	GetHandleVerifier [0x00A37163+2416035]
	GetHandleVerifier [0x00A5350C+2531660]
	GetHandleVerifier [0x0080F1B5+155125]
	GetHandleVerifier [0x00815B5D+182173]
	GetHandleVerifier [0x007FF9B8+91640]
	GetHandleVerifier [0x007FFB60+92064]
	GetHandleVerifier [0x007EA620+4704]
	Ba

1. API를 통한 데이터 수집 : naver, 공공데이터
2. requests get
3. 정적웹페이지 데이터 수집 : yes24, 상장정보, naver금융 -> requests.get, requests.post
4. 동적웹페이지 - naver 책, 하나은행 환율정보, 구글play 리뷰 -> selenium
5. 함수화, 모듈화(dbio), 실행파일로 만들고 스케줄러로 자동화

#### 데이터 파이프라인 : 수집 / 정제 / 저장 + 분석 / 분석결과까지 자동화 가능(분석 결과가 괜찮게 나옴)
#### DB, HTML, CSS, Javascript